In [1]:
import ray
from ray import serve
import requests

/Users/minseokbae/anaconda3/envs/ray-serve-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-25 17:57:16,256	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
serve.start()

2023-09-25 17:57:21,309	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(HTTPProxyActor pid=21093) INFO:     Started server process [21093]


In [4]:
@serve.deployment
def say_hello(request):
    name = request.query_params["name"]
    return f"Hi {name}"

In [5]:
say_hello.deploy()

(ServeController pid=21087) INFO 2023-09-25 17:58:37,406 controller 21087 deployment_state.py:1308 - Deploying new version of deployment say_hello.
(ServeController pid=21087) INFO 2023-09-25 17:58:37,410 controller 21087 deployment_state.py:1571 - Adding 1 replica to deployment say_hello.


In [6]:
response = requests.get("http://127.0.0.1:8000/say_hello?name=Michael").text
print(response)

Hi Michael


(ServeReplica:say_hello pid=21277) INFO 2023-09-25 18:00:41,548 say_hello say_hello#NFUHyv FitUKLSWMt /say_hello replica.py:723 - __CALL__ OK 6.1ms


In [7]:
from transformers import AutoModel

model_name = "roberta-base"
model = AutoModel.from_pretrained(model_name, device_map="auto")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
%pip install optimum
# Convert the model into Better Transformer format to run faster
from optimum.bettertransformer import BetterTransformer
bt_model = BetterTransformer.transform(model, keep_original_model=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 7.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/09/7e/fd4d6441a541dba61d0acb3c1fd5df53214c2e9033854e837a99dd9e0793/datasets-2.14.5-py3-none-any.whl.metadata
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
  Obtaining dependency information for pyarrow>=8.0.0 from https://files.pythonhosted.org/packages/04/3d/183285e5f038866b25e35c9f586abafd78f8deb88d783d313a85f03fc403/pyarrow-13.0.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Using cached dil

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [10]:
# Use pipeline to run the model

from optimum.pipelines import pipeline
unmasked = pipeline("fill-mask", "distilbert-base-uncased", accelerator="bettertransformer")
unmasked("My favorite animal is a [MASK]")

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
/Users/minseokbae/anaconda3/envs/ray-serve-env/lib/python3.9/site-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:179.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


[{'score': 0.07488887012004852,
  'token': 20325,
  'token_str': 'donkey',
  'sequence': 'my favorite animal is a donkey'},
 {'score': 0.07192376255989075,
  'token': 10442,
  'token_str': 'rabbit',
  'sequence': 'my favorite animal is a rabbit'},
 {'score': 0.05241290107369423,
  'token': 1012,
  'token_str': '.',
  'sequence': 'my favorite animal is a.'},
 {'score': 0.0469793938100338,
  'token': 8448,
  'token_str': 'deer',
  'sequence': 'my favorite animal is a deer'},
 {'score': 0.03875979781150818,
  'token': 18197,
  'token_str': 'squirrel',
  'sequence': 'my favorite animal is a squirrel'}]

In [11]:
# define an inference function that uses the pipeline
@serve.deployment
def masked(request):
    return unmasked(request.query_params["text"])

In [12]:
masked.deploy()

(ServeController pid=21087) INFO 2023-09-25 18:21:15,008 controller 21087 deployment_state.py:1308 - Deploying new version of deployment masked.
(ServeController pid=21087) INFO 2023-09-25 18:21:15,043 controller 21087 deployment_state.py:1571 - Adding 1 replica to deployment masked.
(ServeController pid=21087) WARNING 2023-09-25 18:21:45,300 controller 21087 deployment_state.py:1882 - Deployment "masked" has 1 replicas that have taken more than 30s to be scheduled. This may be caused by waiting for the cluster to auto-scale, or waiting for a runtime environment to install. Resources required for each replica: {"CPU": 1}, resources available: {"CPU": 6.0}.


In [13]:
response = requests.get("http://127.0.0.1:8000/masked?text=My favorite animal is a [MASK]").text
print(response)


(ServeReplica:masked pid=24673) /Users/minseokbae/anaconda3/envs/ray-serve-env/lib/python3.9/site-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:179.)
(ServeReplica:masked pid=24673)   hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


[{"score": 0.07488887012004852, "token": 20325, "token_str": "donkey", "sequence": "my favorite animal is a donkey"}, {"score": 0.07192376255989075, "token": 10442, "token_str": "rabbit", "sequence": "my favorite animal is a rabbit"}, {"score": 0.05241290107369423, "token": 1012, "token_str": ".", "sequence": "my favorite animal is a."}, {"score": 0.0469793938100338, "token": 8448, "token_str": "deer", "sequence": "my favorite animal is a deer"}, {"score": 0.03875979781150818, "token": 18197, "token_str": "squirrel", "sequence": "my favorite animal is a squirrel"}]


(ServeReplica:masked pid=24673) INFO 2023-09-25 18:23:20,072 masked masked#SuJmlA sbHAscMQcx /masked replica.py:723 - __CALL__ OK 1028.7ms


In [14]:
response2 = requests.get("http://127.0.0.1:8000/masked?text=I go to [MASK] university.").text
print(response2)

[{"score": 0.05190018564462662, "token": 5765, "token_str": "harvard", "sequence": "i go to harvard university."}, {"score": 0.043821074068546295, "token": 8422, "token_str": "stanford", "sequence": "i go to stanford university."}, {"score": 0.042906805872917175, "token": 7996, "token_str": "yale", "sequence": "i go to yale university."}, {"score": 0.03194918856024742, "token": 10921, "token_str": "cornell", "sequence": "i go to cornell university."}, {"score": 0.02128937654197216, "token": 9173, "token_str": "princeton", "sequence": "i go to princeton university."}]


(ServeReplica:masked pid=24673) INFO 2023-09-25 18:24:40,211 masked masked#SuJmlA vDvTEBVPPn /masked replica.py:723 - __CALL__ OK 2093.4ms
